In [12]:
import pandas as pd
import re

In [13]:
# Load the provided file
file_path = 'source_file_GIB.csv'
data_raw = pd.read_csv(file_path, delimiter='|')

In [14]:
## Exclude non-prescription drugs (Only keep UITSLUITEND RECEPT)

data = data_raw.copy()

data['AFLEVERSTATUS_CLEAN'] = data['AFLEVERSTATUS'].apply(
    lambda x: x.split('$SEMICOLON$ ') if isinstance(x, str) else []
)

data = data[data['AFLEVERSTATUS_CLEAN'].apply(lambda x: 'Uitsluitend recept' in x)]


In [15]:
# Function to extract various strength formats, including concentrations, percentages, and ratios
def optimized_extract_strength(product_name):    
    units = r'ppm|mg| mg|miligram|g|gram|ml|mililiter|µg|microgram|microgam|mcg|IE|I.E|internationale eenheden|eenheden|IU|I.U|SQ-E|anti-Xa-eenheden|dosis|ME|uur|%|miljoen internationale eenheden|mmol|mol|l|liter|Bq|MBq|SQ-T|-SQ-T|DU|AU|Allerganeenheden|miljoen E|miljoen IE|miljoen I.E.|E|U|IR|vg|SQ-HDM|SQ-Bet|BU|TE|microliter|sferoïden|cm²|kBq|KIE|CFU|plaquevormende eenheden (PFU)/ml| PFU|plaquevormende eenheden|MU|SBE|anti-heparine IE|Speywood-eenheden|cellen'
    
    match=re.search(rf'(\d+([.,]?)\d*)(\/\d+([.,]?)\d*)*\s?({units})\s?(\/\s?\d*([.,]?)\d*\s?({units}))*',product_name, re.IGNORECASE)


    if match:
        return match.group(0).strip()  # Clean up any extra spaces
    else:
        return None
# Apply the optimized function to extract strength from PRODUCTNAAM
data['STERKTE'] = data.apply(
    lambda row: optimized_extract_strength(row['PRODUCTNAAM']),
    axis=1
)



#####################
### Standardize units ###
unit_mapping = {
    'microgram':'mcg',
    'microgam':'mcg',
    'µg':'mcg',
    'miligram':'mg',
    'gram':'g',
    'mililiter':'ml',
    'I.E.':'IE',
    'internationale eenheden': 'IE',
    'IU':'IE',
    'I.U.':'IE',
    'miljoen IE': 'miljoen IE',
    'miljoen I.E.': 'miljoen IE',
    'miljoen internationale eenheden':'miljoen IE',
    'miljoen E': 'miljoen eenheden',
    'ME': 'miljoen eenheden',
    'E': 'eenheden',
    'U': 'eenheden',
    'plaquevormende eenheden (PFU)/ml': 'PFU',
    'plaquevormende eenheden': 'PFU',

    # Add more mappings as needed
}

def standardize_unit(sterkte):
    for key, value in unit_mapping.items():
        # Use regex to ensure we match whole words to avoid partial replacements
        pattern = r'\b' + re.escape(key) + r'\b'
        try:
            if re.search(pattern, sterkte, re.IGNORECASE):
                return re.sub(pattern, value, sterkte, flags=re.IGNORECASE)
        except:
            return sterkte
    return sterkte  # Return original if no mapping found



data['STERKTE_STANDARDIZED'] = data['STERKTE'].apply(standardize_unit)


### Replace decimal points by komma's
def standardize_decimals(sterkte):
    pattern_decimal='[.]'
    pattern_thousand_sep='[.]\d3*'
    try:
       if re.search(pattern_thousand_sep, sterkte, re.IGNORECASE):
           return sterkte
       elif re.search(pattern_decimal, sterkte, re.IGNORECASE):
           return re.sub(pattern_decimal, ',', sterkte, flags=re.IGNORECASE)
    except:
        return sterkte
    
    return sterkte

data['STERKTE_STANDARDIZED'] = data['STERKTE_STANDARDIZED'].apply(standardize_decimals)


temp = data[['PRODUCTNAAM','STERKTE','STERKTE_STANDARDIZED']]
#temp.to_clipboard(excel=True)


In [16]:
data['WERKZAMESTOFFEN_CLEAN'] = data['WERKZAMESTOFFEN'].replace(r'#','/', regex=True)


# Define a list of common salts and hydration terms in Dutch
common_salts_dutch = ["(di)?hydrochloride", "nitraat", "fosfaat", "natrium", "kalium", "calcium", "magnesium",
                      "sulfaat", "tartraat", "mesilaat", "acetaat", "fumaraat", "malaat", "chloride", 
                      "bromide", "succinaat", "mesylaat","propionaat",'xinafoaat',"butiraat","butyraat", "bes[iy]laat","arginine",
                      "(un)?decanoaat", "maleaat", "citraat","diwaterstof", "valeraat", "lactobionaat","benzoaat",
                      "(-)?(TERT)?(-)?BUTYLAMINE", "CILEXETIL", "PROPYLEENGLYCOLAAT", "etexilaat", "carbonaat",
                      "oxalaat", "--", "pivalaat"
                      ]

# Exclusion list for words/phrases you want to keep intact
exclusion_list = set(["carbasalaatcalcium"])

# Compile the regex pattern for salts
salt_pattern = re.compile(r'(' + '|(di|hydro|X-|waterstof)?'.join(common_salts_dutch) + r')', re.IGNORECASE)

# Regex to match numbers followed by '-water'
water_pattern = re.compile(r'((\d+([.,]?)\d*)|n|X)-water', re.IGNORECASE)

# Generalized function to remove unwanted patterns (salts and '-water') while respecting the exclusion list
def clean_ingredient(ingredient):
    word = ingredient#.split()
    
    cleaned_words = []

    word_lower = word.lower()  # Use lowercase for exclusion comparison
        
    if word_lower in exclusion_list:
        cleaned_words.append(word)  # Skip cleaning for excluded words
    else:
        # Remove salts and '-water' patterns
        cleaned_word = salt_pattern.sub('', word)
        cleaned_word = water_pattern.sub('', cleaned_word)
        cleaned_words.append(cleaned_word.strip())  # Add the cleaned word
    
    return ' '.join(cleaned_words).strip()


# Apply the function to remove salts and hydration terms
data['WERKZAMESTOFFEN_CLEAN'] = data['WERKZAMESTOFFEN_CLEAN'].fillna('').apply(clean_ingredient)

for n in range(2):
    data['WERKZAMESTOFFEN_CLEAN'] = data['WERKZAMESTOFFEN_CLEAN'].replace(r'\/ ', r'\/', regex=True)
    data['WERKZAMESTOFFEN_CLEAN'] = data['WERKZAMESTOFFEN_CLEAN'].replace(r' \/', r'\/', regex=True)
    data['WERKZAMESTOFFEN_CLEAN'] = data['WERKZAMESTOFFEN_CLEAN'].str.replace(r'\\/', '/', regex=True)

def remove_duplicate_ingredients(s):
    ingredients = s.split('/')
    unique_ingredients = []
    [unique_ingredients.append(i) for i in ingredients if i not in unique_ingredients]
    return '/'.join(unique_ingredients)

# Apply the function to the 'Name' column
data['WERKZAMESTOFFEN_CLEAN'] = data['WERKZAMESTOFFEN_CLEAN'].apply(remove_duplicate_ingredients)

temp = data['WERKZAMESTOFFEN_CLEAN'].drop_duplicates()

temp.to_clipboard(excel=True, index=False, sep=';')


In [17]:
data['FARMACEUTISCHEVORM_CLEAN'] = data['FARMACEUTISCHEVORM'].fillna("").apply(lambda x: x.split(',')[0].strip())

temp = data.groupby(['FARMACEUTISCHEVORM_CLEAN'])['FARMACEUTISCHEVORM_CLEAN'].count().reset_index(name='counts')
temp.to_clipboard(excel=True, index=False, sep=';')


In [18]:
#Drop duplicates, based on handelsproduct
data = data.drop_duplicates(subset=['PRODUCTNAAM'], keep='first')

In [19]:
data.columns

Index(['REGISTRATIENUMMER', 'SOORT', 'PRODUCTNAAM', 'INSCHRIJVINGSDATUM',
       'HANDELSVERGUNNINGHOUDER', 'AFLEVERSTATUS', 'FARMACEUTISCHEVORM',
       'POTENTIE', 'PROCEDURENUMMER', 'TOEDIENINGSWEG',
       'AANVULLENDEMONITORING', 'SMPC_FILENAAM', 'BIJSLUITER_FILENAAM',
       'PAR_FILENAAM', 'SPAR_FILENAAM', 'ARMM_FILENAAM', 'SMPC_WIJZIG_DATUM',
       'BIJSLUITER_WIJZIG_DATUM', 'ATC', 'WERKZAMESTOFFEN', 'HULPSTOFFEN',
       'PRODUCTDETAIL_LINK', 'NIEUWS_LINKS', 'NIEUWS_LINK_DATUMS',
       'REFERENTIE', 'SMPC_VORIGE_VERSIE', 'SMPC_VORIGE_VORIGE_VERSIE',
       'AFLEVERSTATUS_CLEAN', 'STERKTE', 'STERKTE_STANDARDIZED',
       'WERKZAMESTOFFEN_CLEAN', 'FARMACEUTISCHEVORM_CLEAN'],
      dtype='object')

In [20]:
data_clean = data[['PRODUCTNAAM','WERKZAMESTOFFEN_CLEAN','STERKTE_STANDARDIZED','FARMACEUTISCHEVORM_CLEAN' ]]

In [23]:
data_final = data_clean.rename(columns={'PRODUCTNAAM':'Brandname','WERKZAMESTOFFEN_CLEAN':'ActiveIngredient','STERKTE_STANDARDIZED':'Strength','FARMACEUTISCHEVORM_CLEAN':'DosageForm'})
data_final['PrescribingProduct'] = data_final['ActiveIngredient'].fillna("") + " - " + data_final['Strength'].fillna("") + " - " + data_final['DosageForm'].fillna("")  


#Drop rows without strength
data_final = data_final.dropna(axis=0)


temp=data_final.head(20)
temp.to_clipboard(excel=True, index=False)

In [24]:
import os
import pandas as pd
import logging
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, UniqueConstraint
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
from sqlalchemy.exc import IntegrityError

# -----------------------------------
# Configuration and Setup
# -----------------------------------

# Configure logging
logging.basicConfig(
    filename='ehr_database_setup.log',
    level=logging.INFO,
    format='%(asctime)s:%(levelname)s:%(message)s'
)

# Define the base class for declarative class definitions
Base = declarative_base()

# -----------------------------------
# Define SQLAlchemy Models
# -----------------------------------

class ActiveIngredient(Base):
    __tablename__ = 'active_ingredients'
    IngredientID = Column(Integer, primary_key=True, autoincrement=True)
    ActiveIngredientName = Column(String, unique=True, nullable=False)
    prescribing_products = relationship('PrescribingProduct', back_populates='active_ingredient')

class DosageForm(Base):
    __tablename__ = 'dosage_forms'
    DosageFormID = Column(Integer, primary_key=True, autoincrement=True)
    DosageFormDescription = Column(String, unique=True, nullable=False)
    prescribing_products = relationship('PrescribingProduct', back_populates='dosage_form')

class PrescribingProduct(Base):
    __tablename__ = 'prescribing_products'
    ProductID = Column(Integer, primary_key=True, autoincrement=True)
    ActiveIngredientID = Column(Integer, ForeignKey('active_ingredients.IngredientID'), nullable=False)
    DosageFormID = Column(Integer, ForeignKey('dosage_forms.DosageFormID'), nullable=False)
    PrescribingProduct = Column(String, nullable=False)  # Product name or identifier
    Strength = Column(String, nullable=False)
    
    active_ingredient = relationship('ActiveIngredient', back_populates='prescribing_products')
    dosage_form = relationship('DosageForm', back_populates='prescribing_products')
    brands = relationship('Brand', back_populates='prescribing_product')

    __table_args__ = (
        UniqueConstraint('ActiveIngredientID', 'DosageFormID', 'PrescribingProduct', 'Strength', name='_prescribing_product_uc'),
    )

class Brand(Base):
    __tablename__ = 'brands'
    BrandID = Column(Integer, primary_key=True, autoincrement=True)
    BrandName = Column(String, nullable=False)
    ProductID = Column(Integer, ForeignKey('prescribing_products.ProductID'), nullable=False)
    
    prescribing_product = relationship('PrescribingProduct', back_populates='brands')

    __table_args__ = (
        UniqueConstraint('BrandName', 'ProductID', name='_brand_uc'),
    )

# -----------------------------------
# Define Helper Functions
# -----------------------------------

def get_or_create(session, model, **kwargs):
    """
    Helper function to get an instance or create it if it doesn't exist.
    """
    instance = session.query(model).filter_by(**kwargs).first()
    if instance:
        return instance
    else:
        instance = model(**kwargs)
        session.add(instance)
        try:
            session.commit()
            logging.info(f"Created new {model.__tablename__}: {kwargs}")
        except IntegrityError as e:
            session.rollback()
            logging.error(f"IntegrityError when creating {model.__tablename__}: {kwargs} - {e}")
            return None
        return instance

# -----------------------------------
# Main Execution
# -----------------------------------

def main():
    # Assume 'data_final' is your preprocessed DataFrame
    # Ensure that 'data_final' is defined before running this script
    # For example:
    # data_final = pd.read_csv('your_preprocessed_data.csv')

    # Check if 'data_final' exists
    if 'data_final' not in globals():
        logging.error("DataFrame 'data_final' is not defined.")
        print("Error: DataFrame 'data_final' is not defined. Please ensure it is loaded before running the script.")
        return

    # Check for missing or empty 'Strength' values
    missing_strength = data_final[data_final['Strength'].isnull() | (data_final['Strength'].str.strip() == '')]

    if not missing_strength.empty:
        logging.warning("Rows with missing or empty Strength:")
        logging.warning(missing_strength[['Brandname', 'Strength']])
        print("Rows with missing or empty Strength found. Check the log for details.")
        # Depending on requirements, you might want to exit or proceed
        return
    else:
        logging.info("No missing or empty Strength values found.")
        print("No missing or empty Strength values found.")

    # Extract unique entries for ActiveIngredients and DosageForms
    unique_ingredients = data_final['ActiveIngredient'].unique()
    unique_dosage_forms = data_final['DosageForm'].unique()
    unique_prescribing_products = data_final[['PrescribingProduct', 'ActiveIngredient', 'DosageForm', 'Strength']].drop_duplicates()

    # Define the path to the SQLite database
    db_path = 'ehr_medications.db'

    # Check if the database file exists and delete it
    if os.path.exists(db_path):
        try:
            os.remove(db_path)
            logging.info(f"Deleted existing database file: {db_path}")
            print(f"Deleted existing database file: {db_path}")
        except PermissionError as e:
            logging.error(f"PermissionError when deleting the database file: {e}")
            print(f"PermissionError: Cannot delete '{db_path}' because it is being used by another process.")
            print("Please ensure no other applications are accessing the database and try again.")
            print("Close any SQLite browsers or other applications that might be using the database file.")
            print("If the issue persists, consider restarting your computer to release file locks.")
            return
    else:
        logging.info(f"No existing database file found at: {db_path}")
        print(f"No existing database file found at: {db_path}")

    # Create a SQLite database
    engine = create_engine(f'sqlite:///{db_path}', echo=False)

    # Create all tables in the engine
    Base.metadata.create_all(engine)
    logging.info("All tables have been created.")
    print("All tables have been created.")

    # Create a configured "Session" class
    Session = sessionmaker(bind=engine)

    # Create a Session using a context manager to ensure closure
    with Session() as session:
        # -----------------------------------
        # Insert ActiveIngredients
        # -----------------------------------
        print("Inserting Active Ingredients...")
        for ingredient in unique_ingredients:
            get_or_create(session, ActiveIngredient, ActiveIngredientName=ingredient)

        # -----------------------------------
        # Insert DosageForms
        # -----------------------------------
        print("Inserting Dosage Forms...")
        for form in unique_dosage_forms:
            get_or_create(session, DosageForm, DosageFormDescription=form)

        # -----------------------------------
        # Insert PrescribingProducts
        # -----------------------------------
        print("Inserting Prescribing Products...")
        for _, row in unique_prescribing_products.iterrows():
            prescribing_product_name = row['PrescribingProduct']
            active_ingredient_name = row['ActiveIngredient']
            dosage_form_description = row['DosageForm']
            strength = row['Strength']

            # Get ActiveIngredientID
            active_ingredient = session.query(ActiveIngredient).filter_by(ActiveIngredientName=active_ingredient_name).first()
            if not active_ingredient:
                logging.error(f"ActiveIngredient '{active_ingredient_name}' not found for PrescribingProduct '{prescribing_product_name}'.")
                print(f"Error: ActiveIngredient '{active_ingredient_name}' not found for PrescribingProduct '{prescribing_product_name}'. Skipping.")
                continue

            # Get DosageFormID
            dosage_form = session.query(DosageForm).filter_by(DosageFormDescription=dosage_form_description).first()
            if not dosage_form:
                logging.error(f"DosageForm '{dosage_form_description}' not found for PrescribingProduct '{prescribing_product_name}'.")
                print(f"Error: DosageForm '{dosage_form_description}' not found for PrescribingProduct '{prescribing_product_name}'. Skipping.")
                continue

            # Check if PrescribingProduct already exists with the same ActiveIngredientID, DosageFormID, PrescribingProduct name, and Strength
            existing_product = session.query(PrescribingProduct).filter_by(
                ActiveIngredientID=active_ingredient.IngredientID,
                DosageFormID=dosage_form.DosageFormID,
                PrescribingProduct=prescribing_product_name,
                Strength=strength
            ).first()

            if existing_product:
                logging.info(f"PrescribingProduct '{prescribing_product_name}' with Strength '{strength}' already exists with ProductID: {existing_product.ProductID}.")
                continue

            # Create PrescribingProduct
            prescribing_product = PrescribingProduct(
                ActiveIngredientID=active_ingredient.IngredientID,
                DosageFormID=dosage_form.DosageFormID,
                PrescribingProduct=prescribing_product_name,
                Strength=strength
            )
            session.add(prescribing_product)
            try:
                session.commit()
                logging.info(f"Inserted PrescribingProduct: {prescribing_product_name} with Strength: {strength}")
                print(f"Inserted PrescribingProduct: {prescribing_product_name} with Strength: {strength}")
            except IntegrityError as e:
                session.rollback()
                logging.error(f"IntegrityError inserting PrescribingProduct '{prescribing_product_name}' with Strength '{strength}': {e}")
                print(f"IntegrityError inserting PrescribingProduct '{prescribing_product_name}' with Strength '{strength}': {e}")
            except Exception as e:
                session.rollback()
                logging.error(f"Error inserting PrescribingProduct '{prescribing_product_name}' with Strength '{strength}': {e}")
                print(f"Error inserting PrescribingProduct '{prescribing_product_name}' with Strength '{strength}': {e}")

        # -----------------------------------
        # Insert Brands
        # -----------------------------------
        print("Inserting Brands...")
        # Iterate through each row in data_final to insert Brands
        for index, row in data_final.iterrows():
            brand_name = row['Brandname']
            strength = row['Strength']
            prescribing_product_name = row['PrescribingProduct']

            # Find the corresponding PrescribingProduct with matching Strength
            prescribing_product = session.query(PrescribingProduct).filter_by(
                PrescribingProduct=prescribing_product_name,
                Strength=strength
            ).first()
            if not prescribing_product:
                logging.error(f"PrescribingProduct '{prescribing_product_name}' with Strength '{strength}' not found for Brand '{brand_name}'.")
                print(f"Error: PrescribingProduct '{prescribing_product_name}' with Strength '{strength}' not found for Brand '{brand_name}'. Skipping.")
                continue

            # Check for duplicate Brand entry
            existing_brand = session.query(Brand).filter_by(
                BrandName=brand_name,
                ProductID=prescribing_product.ProductID
            ).first()

            if existing_brand:
                logging.warning(f"Duplicate Brand found at index {index}: Brand='{brand_name}', PrescribingProduct='{prescribing_product_name}', Strength='{strength}'. Skipping.")
                print(f"Duplicate Brand found at index {index}: Brand='{brand_name}', PrescribingProduct='{prescribing_product_name}', Strength='{strength}'. Skipping.")
                continue

            # Create Brand
            brand = Brand(
                BrandName=brand_name,
                ProductID=prescribing_product.ProductID
            )
            session.add(brand)
            try:
                session.commit()
                logging.info(f"Inserted Brand: {brand_name} linked to PrescribingProduct '{prescribing_product_name}' with Strength '{strength}'")
                print(f"Inserted Brand: {brand_name} linked to PrescribingProduct '{prescribing_product_name}' with Strength '{strength}'")
            except IntegrityError as e:
                session.rollback()
                logging.error(f"IntegrityError inserting Brand '{brand_name}' linked to PrescribingProduct '{prescribing_product_name}': {e}")
                print(f"IntegrityError inserting Brand '{brand_name}' linked to PrescribingProduct '{prescribing_product_name}': {e}")
            except Exception as e:
                session.rollback()
                logging.error(f"Error inserting Brand '{brand_name}' linked to PrescribingProduct '{prescribing_product_name}': {e}")
                print(f"Error inserting Brand '{brand_name}' linked to PrescribingProduct '{prescribing_product_name}': {e}")

    print("Relational database 'ehr_medications.db' has been created and populated successfully.")

# -----------------------------------
# Execute the Script
# -----------------------------------

if __name__ == "__main__":
    main()


No missing or empty Strength values found.
No existing database file found at: ehr_medications.db
All tables have been created.
Inserting Active Ingredients...
Inserting Dosage Forms...
Inserting Prescribing Products...
Inserted PrescribingProduct: DEXAMETHASON - 1 mg/ml - Oogdruppels with Strength: 1 mg/ml
Inserted PrescribingProduct: AMYLASE/LIPASE/PANCREATINE/PROTEASE - 25.000 eenheden - Maagsapresistente capsule with Strength: 25.000 eenheden
Inserted PrescribingProduct: DEFEROXAMINE - 500 mg - Poeder voor oplossing voor injectie of infusie with Strength: 500 mg
Inserted PrescribingProduct: ESOMEPRAZOL/NAPROXEN - 500 mg/20 mg - Tablet met gereguleerde afgifte with Strength: 500 mg/20 mg
Inserted PrescribingProduct: TRIPTORELINEPAMOAAT - 22,5 mg - Poeder en oplosmiddel voor suspensie voor injectie met verlengde afgifte with Strength: 22,5 mg
Inserted PrescribingProduct: ENOXAPARINE - 10.000 IE - Oplossing voor injectie with Strength: 10.000 IE
Inserted PrescribingProduct: ROSUVASTAT